In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 4
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_Canny&Pyramid_L_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_f1 = np.load('../trainImage_canny.npy')
        self.test_x_f1 = np.load('../testImage_canny.npy')
        self.train_x_f2 = np.load('../trainImage_pyramid_L.npy')
        self.test_x_f2 = np.load('../testImage_pyramid_L.npy')
        self.train_x_f3 = np.load('../trainImage_pyramid_G.npy')
        self.test_x_f3 = np.load('../testImage_pyramid_G.npy')

        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_f1, x_f2, x_f3, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_f1",x_f1.get_shape()) 
            print("CNN:x_f2",x_f2.get_shape())  
            print("CNN:x_f3",x_f3.get_shape())  
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x_f1,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_f1,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())
            
            #输入x_f2,卷积核为3*3 输出维度为32
            net1_3 = tf.layers.conv2d(inputs = x_f2,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_3'               # 命名用于获取变量
                                    )
            print("CNN:",net1_3.get_shape())
            
            #输入x_f3,卷积核为3*3 输出维度为32
            net1_4 = tf.layers.conv2d(inputs = x_f3,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_4'               # 命名用于获取变量
                                    )
            print("CNN:",net1_4.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.concat([net, net1_3], 3)
            net = tf.concat([net, net1_4], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_f1 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f1.shape[1], 
                                                     self.train_x_f1.shape[2], self.train_x_f1.shape[3]],
                                  name='x_f1')
        self.x_f2 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f2.shape[1], 
                                                     self.train_x_f2.shape[2], self.train_x_f2.shape[3]],
                                  name='x_f2')
        self.x_f3 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f3.shape[1], 
                                             self.train_x_f3.shape[2], self.train_x_f3.shape[3]],
                          name='x_f3')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_f1, self.x_f2, self.x_f3, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x,  self.x_f1, self.x_f2, self.x_f3, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_f1 = self.train_x_f1[shuffle_idxs]
            shuffled_set_f2 = self.train_x_f2[shuffle_idxs]
            shuffled_set_f3 = self.train_x_f3[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f1 = shuffled_set_f1[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f2 = shuffled_set_f2[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f3 = shuffled_set_f3[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_f1: batch_x_f1,
                                                          self.x_f2: batch_x_f2,
                                                          self.x_f3: batch_x_f3,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f3_test =self.test_x_f3[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_f1: batch_x_f1_test,
                                                                    self.x_f2: batch_x_f2_test,
                                                                    self.x_f3: batch_x_f3_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f3_test =self.test_x_f3[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.x_f3: batch_x_f3_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [3]:
dataset = '4_Flowers_1s'
epoch = 80
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
# Avg per epoch ptime: 18.71, total 80 epochs ptime: 1530.40
#  [*] Training finished!
#  [*] Best Epoch:  70 , Accuracy:  0.753333330154419
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay-70
#  [*] Finished testing Best Epoch: 70 , accuracy:  0.753333330154419 !

CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN:x_f3 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 128)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 4)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN:x_f3 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 128)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 4)
------------------------
---------
Variables: name (type shape) [

Epoch: [ 3] [  10/  40] time: 47.9799, loss: 0.04585519
Epoch: [ 3] [  11/  40] time: 48.4197, loss: 0.04187297
Epoch: [ 3] [  12/  40] time: 48.8545, loss: 0.09252308
Epoch: [ 3] [  13/  40] time: 49.2929, loss: 0.04318524
Epoch: [ 3] [  14/  40] time: 49.7314, loss: 0.09327544
Epoch: [ 3] [  15/  40] time: 50.1717, loss: 0.15901861
Epoch: [ 3] [  16/  40] time: 50.6131, loss: 0.07226137
Epoch: [ 3] [  17/  40] time: 51.0524, loss: 0.04967514
Epoch: [ 3] [  18/  40] time: 51.4967, loss: 0.11542899
Epoch: [ 3] [  19/  40] time: 51.9347, loss: 0.09343287
Epoch: [ 3] [  20/  40] time: 52.3812, loss: 0.04180147
Epoch: [ 3] [  21/  40] time: 52.8209, loss: 0.12913743
Epoch: [ 3] [  22/  40] time: 53.2598, loss: 0.03772851
Epoch: [ 3] [  23/  40] time: 53.6955, loss: 0.03810799
Epoch: [ 3] [  24/  40] time: 54.1358, loss: 0.16657898
Epoch: [ 3] [  25/  40] time: 54.5738, loss: 0.07484082
Epoch: [ 3] [  26/  40] time: 55.0128, loss: 0.04814687
Epoch: [ 3] [  27/  40] time: 55.4555, loss: 0.0

Epoch: [ 6] [  33/  40] time: 116.4893, loss: 0.04657619
Epoch: [ 6] [  34/  40] time: 116.9304, loss: 0.02597314
Epoch: [ 6] [  35/  40] time: 117.3726, loss: 0.02489564
Epoch: [ 6] [  36/  40] time: 117.8119, loss: 0.01606329
Epoch: [ 6] [  37/  40] time: 118.2522, loss: 0.04183540
Epoch: [ 6] [  38/  40] time: 118.6949, loss: 0.01942630
Epoch: [ 6] [  39/  40] time: 119.1345, loss: 0.02616533
[6/80] - ptime: 18.9756 loss: 0.06653985 acc: 0.22000 lr: 0.00100000
Epoch: [ 7] [   0/  40] time: 121.5483, loss: 0.03875503
Epoch: [ 7] [   1/  40] time: 121.9908, loss: 0.00986847
Epoch: [ 7] [   2/  40] time: 122.4319, loss: 0.01985372
Epoch: [ 7] [   3/  40] time: 122.8707, loss: 0.03456931
Epoch: [ 7] [   4/  40] time: 123.3120, loss: 0.03752318
Epoch: [ 7] [   5/  40] time: 123.7529, loss: 0.01541811
Epoch: [ 7] [   6/  40] time: 124.1926, loss: 0.07749278
Epoch: [ 7] [   7/  40] time: 124.6379, loss: 0.03364353
Epoch: [ 7] [   8/  40] time: 125.0782, loss: 0.08699562
Epoch: [ 7] [   9/ 

Epoch: [10] [  12/  40] time: 185.2948, loss: 0.02765642
Epoch: [10] [  13/  40] time: 185.7392, loss: 0.04208781
Epoch: [10] [  14/  40] time: 186.1788, loss: 0.02860841
Epoch: [10] [  15/  40] time: 186.6194, loss: 0.00217018
Epoch: [10] [  16/  40] time: 187.0593, loss: 0.04354375
Epoch: [10] [  17/  40] time: 187.5028, loss: 0.03633785
Epoch: [10] [  18/  40] time: 187.9416, loss: 0.01153106
Epoch: [10] [  19/  40] time: 188.3788, loss: 0.00647016
Epoch: [10] [  20/  40] time: 188.8226, loss: 0.04108500
Epoch: [10] [  21/  40] time: 189.2634, loss: 0.02046268
Epoch: [10] [  22/  40] time: 189.7087, loss: 0.02248177
Epoch: [10] [  23/  40] time: 190.1488, loss: 0.05035493
Epoch: [10] [  24/  40] time: 190.5847, loss: 0.03462934
Epoch: [10] [  25/  40] time: 191.0261, loss: 0.02728890
Epoch: [10] [  26/  40] time: 191.4802, loss: 0.00274211
Epoch: [10] [  27/  40] time: 191.9267, loss: 0.04676547
Epoch: [10] [  28/  40] time: 192.3708, loss: 0.00922970
Epoch: [10] [  29/  40] time: 1

Epoch: [13] [  33/  40] time: 253.0831, loss: 0.10213557
Epoch: [13] [  34/  40] time: 253.5237, loss: 0.03000796
Epoch: [13] [  35/  40] time: 253.9678, loss: 0.03137236
Epoch: [13] [  36/  40] time: 254.4108, loss: 0.02450492
Epoch: [13] [  37/  40] time: 254.8526, loss: 0.02455176
Epoch: [13] [  38/  40] time: 255.2885, loss: 0.00520921
Epoch: [13] [  39/  40] time: 255.7287, loss: 0.01828916
[13/80] - ptime: 18.9953 loss: 0.03288886 acc: 0.38000 lr: 0.00090000
Epoch: [14] [   0/  40] time: 258.0331, loss: 0.06087576
Epoch: [14] [   1/  40] time: 258.4734, loss: 0.00474275
Epoch: [14] [   2/  40] time: 258.9162, loss: 0.02240986
Epoch: [14] [   3/  40] time: 259.3559, loss: 0.03026918
Epoch: [14] [   4/  40] time: 259.7953, loss: 0.07362339
Epoch: [14] [   5/  40] time: 260.2355, loss: 0.05484266
Epoch: [14] [   6/  40] time: 260.6798, loss: 0.01802992
Epoch: [14] [   7/  40] time: 261.1186, loss: 0.05186608
Epoch: [14] [   8/  40] time: 261.5597, loss: 0.01812137
Epoch: [14] [   9/

...

Epoch: [66] [   2/  40] time: 1250.5158, loss: 0.00002500
Epoch: [66] [   3/  40] time: 1250.9504, loss: 0.00001170
Epoch: [66] [   4/  40] time: 1251.3846, loss: 0.00000762
Epoch: [66] [   5/  40] time: 1251.8192, loss: 0.00000665
Epoch: [66] [   6/  40] time: 1252.2532, loss: 0.00003581
Epoch: [66] [   7/  40] time: 1252.6868, loss: 0.00000651
Epoch: [66] [   8/  40] time: 1253.1210, loss: 0.00472290
Epoch: [66] [   9/  40] time: 1253.5544, loss: 0.00005934
Epoch: [66] [  10/  40] time: 1253.9895, loss: 0.00016710
Epoch: [66] [  11/  40] time: 1254.4246, loss: 0.05234363
Epoch: [66] [  12/  40] time: 1254.8592, loss: 0.00046445
Epoch: [66] [  13/  40] time: 1255.2922, loss: 0.01266615
Epoch: [66] [  14/  40] time: 1255.7274, loss: 0.01199468
Epoch: [66] [  15/  40] time: 1256.1613, loss: 0.00141216
Epoch: [66] [  16/  40] time: 1256.5983, loss: 0.00318948
Epoch: [66] [  17/  40] time: 1257.0314, loss: 0.00101801
Epoch: [66] [  18/  40] time: 1257.4651, loss: 0.00018981
Epoch: [66] [ 

Epoch: [69] [  20/  40] time: 1314.6594, loss: 0.01226042
Epoch: [69] [  21/  40] time: 1315.0941, loss: 0.01787217
Epoch: [69] [  22/  40] time: 1315.5272, loss: 0.00232953
Epoch: [69] [  23/  40] time: 1315.9619, loss: 0.06856515
Epoch: [69] [  24/  40] time: 1316.3958, loss: 0.00019127
Epoch: [69] [  25/  40] time: 1316.8245, loss: 0.03997492
Epoch: [69] [  26/  40] time: 1317.2574, loss: 0.01737608
Epoch: [69] [  27/  40] time: 1317.6921, loss: 0.00356808
Epoch: [69] [  28/  40] time: 1318.1256, loss: 0.02244839
Epoch: [69] [  29/  40] time: 1318.5582, loss: 0.00907873
Epoch: [69] [  30/  40] time: 1318.9921, loss: 0.00337503
Epoch: [69] [  31/  40] time: 1319.4255, loss: 0.00182698
Epoch: [69] [  32/  40] time: 1319.8601, loss: 0.00200207
Epoch: [69] [  33/  40] time: 1320.2936, loss: 0.00928558
Epoch: [69] [  34/  40] time: 1320.7301, loss: 0.00107949
Epoch: [69] [  35/  40] time: 1321.1635, loss: 0.00327561
Epoch: [69] [  36/  40] time: 1321.5982, loss: 0.00099080
Epoch: [69] [ 

Epoch: [72] [  38/  40] time: 1378.9355, loss: 0.00147110
Epoch: [72] [  39/  40] time: 1379.3701, loss: 0.00016117
[72/80] - ptime: 18.3920 loss: 0.00170809 acc: 0.69333 lr: 0.00047830
Epoch: [73] [   0/  40] time: 1381.2124, loss: 0.00000753
Epoch: [73] [   1/  40] time: 1381.6470, loss: 0.00130821
Epoch: [73] [   2/  40] time: 1382.0816, loss: 0.00003855
Epoch: [73] [   3/  40] time: 1382.5137, loss: 0.00026040
Epoch: [73] [   4/  40] time: 1382.9472, loss: 0.00002196
Epoch: [73] [   5/  40] time: 1383.3827, loss: 0.00112562
Epoch: [73] [   6/  40] time: 1383.8182, loss: 0.00018445
Epoch: [73] [   7/  40] time: 1384.2503, loss: 0.00040332
Epoch: [73] [   8/  40] time: 1384.6850, loss: 0.00034964
Epoch: [73] [   9/  40] time: 1385.1180, loss: 0.00141825
Epoch: [73] [  10/  40] time: 1385.5515, loss: 0.00034138
Epoch: [73] [  11/  40] time: 1385.9842, loss: 0.00896643
Epoch: [73] [  12/  40] time: 1386.4188, loss: 0.00065024
Epoch: [73] [  13/  40] time: 1386.8555, loss: 0.00050683
Ep

Epoch: [76] [  15/  40] time: 1444.1163, loss: 0.00001548
Epoch: [76] [  16/  40] time: 1444.5510, loss: 0.00108855
Epoch: [76] [  17/  40] time: 1444.9846, loss: 0.00001448
Epoch: [76] [  18/  40] time: 1445.4184, loss: 0.00220753
Epoch: [76] [  19/  40] time: 1445.8497, loss: 0.00006725
Epoch: [76] [  20/  40] time: 1446.2844, loss: 0.00023478
Epoch: [76] [  21/  40] time: 1446.7197, loss: 0.00006052
Epoch: [76] [  22/  40] time: 1447.1552, loss: 0.00029843
Epoch: [76] [  23/  40] time: 1447.5883, loss: 0.00002823
Epoch: [76] [  24/  40] time: 1448.0217, loss: 0.00187935
Epoch: [76] [  25/  40] time: 1448.4617, loss: 0.00085753
Epoch: [76] [  26/  40] time: 1448.8963, loss: 0.00134348
Epoch: [76] [  27/  40] time: 1449.3303, loss: 0.00045209
Epoch: [76] [  28/  40] time: 1449.7665, loss: 0.00006213
Epoch: [76] [  29/  40] time: 1450.1997, loss: 0.00009227
Epoch: [76] [  30/  40] time: 1450.6357, loss: 0.00353313
Epoch: [76] [  31/  40] time: 1451.0690, loss: 0.00010152
Epoch: [76] [ 

Epoch: [79] [  33/  40] time: 1508.2947, loss: 0.00017178
Epoch: [79] [  34/  40] time: 1508.7291, loss: 0.00009973
Epoch: [79] [  35/  40] time: 1509.1594, loss: 0.00013450
Epoch: [79] [  36/  40] time: 1509.5983, loss: 0.00001344
Epoch: [79] [  37/  40] time: 1510.0316, loss: 0.00008557
Epoch: [79] [  38/  40] time: 1510.4646, loss: 0.00028480
Epoch: [79] [  39/  40] time: 1510.8982, loss: 0.00002561
[79/80] - ptime: 18.4200 loss: 0.00024505 acc: 0.70333 lr: 0.00047830
Epoch: [80] [   0/  40] time: 1512.7477, loss: 0.00058085
Epoch: [80] [   1/  40] time: 1513.1802, loss: 0.00008766
Epoch: [80] [   2/  40] time: 1513.6177, loss: 0.00002647
Epoch: [80] [   3/  40] time: 1514.0529, loss: 0.00003905
Epoch: [80] [   4/  40] time: 1514.4893, loss: 0.00001472
Epoch: [80] [   5/  40] time: 1514.9223, loss: 0.00071152
Epoch: [80] [   6/  40] time: 1515.3584, loss: 0.00082433
Epoch: [80] [   7/  40] time: 1515.7926, loss: 0.00003626
Epoch: [80] [   8/  40] time: 1516.2250, loss: 0.00010805
Ep

In [4]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放